In [3]:
#Author: Erik Carlson 
#Description: This Script takes a user provided Right Ascention and Declination as defined on lines 16 and 17 and provides the user with the measurement set identification of the quicklook images
#This script requires the use of Python 3 to run properly 
#Future Changes: Provide the User with Multi-Epoch datasets. Expand to fields that are on the edges of tiles
import urllib
import urllib.request
import re
import csv
import requests


#intake and validate values from the user
a = 0
while a ==0: 
    try: 
        RA = input("Please Enter your Right Ascention in Decimal Format: ") #121.98974
        DEC = input("Please Enter your Right Ascention in Decimal Format: ") #4.54293
        Epoch = input("Please Enter the Requested Epoch (1/2), leave blank if unknown: ")
        RA = float(RA)
        DEC = float(DEC)
        if Epoch == '':
            Epoch = '1'
        Epoch = int(Epoch)
        break
    except: 
        print("Incorrect Values Entered")

with open('Tile_Boundaries.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))

for tile in data:
        Dec_Tile_Start = float(tile[1])
        Dec_Tile_End = float(tile[2])
        Dec_Tile_Center = (Dec_Tile_End - Dec_Tile_Start)/2 + Dec_Tile_Start
        
        RA_Tile_Start = float(tile[3])*15
        RA_Tile_End = float(tile[4])*15
        RA_Tile_Center = (RA_Tile_End - RA_Tile_End)/2 + RA_Tile_Start
        
        if abs(RA_Tile_Center - RA) <= 3.75 and abs(Dec_Tile_Center - DEC) <= 2: 
            tile_id = tile[0]
            VLASS_id = tile[5]
            URL = "https://archive-new.nrao.edu/vlass/quicklook/" + VLASS_id + '/' + tile_id + '/'
            page = requests.get(URL).text
            JName_regex = 'J\d{6}[+]\d{6}[.]\d\d[.]\d{4}\S{3}'
            m = re.search(JName_regex, page)
            if m:
                found_JName = m.group(0)
                full_directory_name = VLASS_id + '.ql.' + tile_id + '.' + found_JName
                URL_New = "https://archive-new.nrao.edu/vlass/quicklook/" + VLASS_id + '/' + tile_id + '/' + full_directory_name + '/casa_pipescript.py'
                try:
                    url = URL_New
                    search_file_for_ms =  urllib.request.urlopen(url)
                    for line in search_file_for_ms:
                        decoded_line = line.decode("utf-8")
                        if decoded_line.find(str(VLASS_id)) != -1:
                            start_value = int(decoded_line.find(str(VLASS_id)))
                            end_value = int(decoded_line.find('.ms'))
                            measurement_set_name = decoded_line[start_value:end_value]
                            print(measurement_set_name)
                except:
                    pass
            



Please Enter your Right Ascention in Decimal Format: 151.85367
Please Enter your Right Ascention in Decimal Format: 58.03429
Please Enter the Requested Epoch (1/2), leave blank if unknown: 


In [2]:
###looped based system for large VLASS Survey 

import urllib
import urllib.request
import re
import csv
import requests
import pandas as pd 

source_requests = pd.read_csv("High_Z_Confirmed.csv")

def unique(list1):
 
    # intilize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    # print list
    for x in unique_list:
        print(x)
    return unique_list






Im_Size = 500
Im_Size_Degrees = Im_Size/3600
g = []

for i in range(len(source_requests)):
    unique_ms_per_source = []
    RA = source_requests["RA"][i]
    DEC = source_requests["DEC"][i]
    RA_Right = RA + Im_Size_Degrees
    RA_Left = RA - Im_Size_Degrees
    Dec_Up = DEC + Im_Size_Degrees
    Dec_Down = DEC - Im_Size_Degrees
    with open('Tile_Boundaries.csv', newline='') as csvfile:
        data = list(csv.reader(csvfile))
    measurement_set_list = []
    for tile in data:
            Dec_Tile_Start = float(tile[1])
            Dec_Tile_End = float(tile[2])
            Dec_Tile_Center = (Dec_Tile_End - Dec_Tile_Start)/2 + Dec_Tile_Start
            RA_Tile_Start = float(tile[3])*15
            RA_Tile_End = float(tile[4])*15
            RA_Tile_Center = (RA_Tile_End - RA_Tile_End)/2 + RA_Tile_Start

            RA_L = [RA_Left, RA_Right,RA_Left,RA_Right, RA, RA, RA, RA_Left, RA_Right]
            DEC_L = [Dec_Up, Dec_Down, Dec_Down, Dec_Up, DEC, Dec_Up, Dec_Down, DEC, DEC]

            for i in range(0,len(RA_L)):
                if RA_L[i] > RA_Tile_Start and RA_L[i] < RA_Tile_End and DEC_L[i] > Dec_Tile_Start and DEC_L[i] < Dec_Tile_End:
                    tile_id = tile[0]
                    VLASS_id = tile[5]
                    URL = "https://archive-new.nrao.edu/vlass/quicklook/" + VLASS_id + '/' + tile_id + '/'
                    page = requests.get(URL).text
                    JName_regex = 'J\d{6}[+-]\d{6}[.]\d\d[.]\d{4}\S{3}'
                    m = re.search(JName_regex, page)
                    if m:
                        found_JName = m.group(0)
                        full_directory_name = VLASS_id + '.ql.' + tile_id + '.' + found_JName
                        URL_New = "https://archive-new.nrao.edu/vlass/quicklook/" + VLASS_id + '/' + tile_id + '/' + full_directory_name + '/casa_pipescript.py'
                        try:
                            url = URL_New
                            search_file_for_ms =  urllib.request.urlopen(url)
                            for line in search_file_for_ms:
                                decoded_line = line.decode("utf-8")
                                if decoded_line.find(str(VLASS_id)) != -1:
                                    start_value = int(decoded_line.find(str(VLASS_id)))
                                    end_value = int(decoded_line.find('.ms'))
                                    measurement_set_name = decoded_line[start_value:end_value]
                                    measurement_set_list.append(measurement_set_name)
                        except:
                            pass
    #print("\nThe unique measurement sets required are:")                    
    g.append(unique(measurement_set_list))

VLASS2.2.sb40507148.eb40987311.59548.184427245375
VLASS2.2.sb40448756.eb40965402.59532.47423184028
VLASS2.1.sb38607339.eb38658209.59109.972473263886
VLASS2.2.sb40876886.eb41018090.59552.53119194444
VLASS2.1.sb38520431.eb38554942.59063.924480127316
VLASS2.1.sb38522718.eb38555682.59065.0341228125
VLASS2.1.sb38513454.eb38563508.59068.92709877314
VLASS2.1.sb38483282.eb38504007.59045.97679461805
VLASS2.2.sb40459052.eb40984997.59542.58195721065
VLASS2.2.sb40566705.eb40773292.59506.62136980324
VLASS2.1.sb38610763.eb38659643.59111.9147512037
VLASS2.1.sb38521729.eb38627577.59093.001901145835
VLASS2.1.sb38504727.eb38659863.59113.352947384265
VLASS2.1.sb38458451.eb38565931.59072.733167094906
VLASS2.1.sb38499285.eb38509861.59047.50492306713
VLASS2.1.sb38500385.eb38635359.59099.351010810184
VLASS2.2.sb41024151.eb41035404.59562.53703290509
VLASS2.2.sb40532036.eb40729161.59504.342741932865
VLASS2.1.sb38619162.eb38631105.59096.06071193287
VLASS2.1.sb38607339.eb38658209.59109.972473263886
VLASS2.2.sb40

In [8]:
print(full_directory_name)

VLASS2.1.ql.T25t08.J092340+573000.10.2048.v1


In [12]:
g

[['VLASS2.1.sb38536469.eb38590168.59084.87988079861'],
 ['VLASS2.1.sb38611939.eb38656872.59108.99831386574'],
 ['VLASS2.1.sb38483282.eb38504007.59045.97679461805'],
 ['VLASS2.2.sb40596977.eb40729013.59502.66564846065'],
 ['VLASS2.1.sb38537053.eb38580096.59080.031941134264'],
 ['VLASS2.2.sb40878034.eb40965466.59533.56291280093'],
 ['VLASS2.1.sb38618574.eb38632962.59098.03251238426'],
 ['VLASS2.1.sb38667356.eb38705720.59131.46099141204'],
 ['VLASS2.2.sb40492002.eb40974915.59538.211577905095'],
 ['VLASS2.2.sb40448756.eb40965402.59532.47423184028',
  'VLASS2.2.sb41026955.eb41063444.59572.434632222226'],
 ['VLASS2.1.sb38483282.eb38504007.59045.97679461805'],
 ['VLASS2.2.sb40592521.eb40626426.59497.65618173611'],
 ['VLASS2.2.sb40448756.eb40965402.59532.47423184028'],
 ['VLASS2.2.sb40525912.eb40962423.59529.26309113426'],
 ['VLASS2.2.sb40888787.eb40985494.59545.33040479167'],
 ['VLASS2.2.sb40492002.eb40974915.59538.211577905095'],
 ['VLASS2.2.sb40490414.eb40974925.59539.17081951389'],
 ['VLAS

In [3]:
import pandas as pd 
df = pd.DataFrame(g)


In [4]:
df

,0,1
0,VLASS2.2.sb40507148.eb40987311.59548.184427245375,None
1,VLASS2.2.sb40448756.eb40965402.59532.47423184028,None
2,VLASS2.1.sb38607339.eb38658209.59109.972473263886,None
3,VLASS2.2.sb40876886.eb41018090.59552.53119194444,None
4,VLASS2.1.sb38520431.eb38554942.59063.924480127316,VLASS2.1.sb38522718.eb38555682.59065.0341228125
...,...,...
125,VLASS2.2.sb40525912.eb40962423.59529.26309113426,None
126,VLASS2.2.sb40875484.eb40911267.59513.63094487268,None
127,VLASS2.1.sb38458451.eb38565931.59072.733167094906,None
128,VLASS2.2.sb40532036.eb40729161.59504.342741932865,None


In [15]:
df.to_csv('no_vla_obs_MSs.csv')

In [16]:
df

,0,1
0,VLASS2.1.sb38536469.eb38590168.59084.87988079861,None
1,VLASS2.1.sb38611939.eb38656872.59108.99831386574,None
2,VLASS2.1.sb38483282.eb38504007.59045.97679461805,None
3,VLASS2.2.sb40596977.eb40729013.59502.66564846065,None
4,VLASS2.1.sb38537053.eb38580096.59080.031941134264,None
5,VLASS2.2.sb40878034.eb40965466.59533.56291280093,None
6,VLASS2.1.sb38618574.eb38632962.59098.03251238426,None
7,VLASS2.1.sb38667356.eb38705720.59131.46099141204,None
8,VLASS2.2.sb40492002.eb40974915.59538.211577905095,None
9,VLASS2.2.sb40448756.eb40965402.59532.47423184028,VLASS2.2.sb41026955.eb41063444.59572.434632222226
